# MAIN

In [1]:
import tensorflow as tf
import numpy as np
import model_tf2.net_factory as netf

In [2]:
tf.test.gpu_device_name()

''

In [3]:
#init some global variables
num_train_files = 128 #number of training tfrecords
num_val_files = 64 #number of testing tfrecords
buffer_size = 100
num_channels = 3
img_size = 64
num_classes = 200

In [7]:
def get_filenames(is_training):
    """
    input: is_training 
    output: a list of training/validation file names
    """
    filenames = []
    if is_training:
        for i in range(num_train_files):
            filename = "data/tf_records/train/" + 'train-%05d-of-00128' % i
            filenames.append(filename)
    else:
        for i in range(num_val_files):
            filename = "data/tf_records/val/" + 'val-%05d-of-00064' % i
            filenames.append(filename)
    return filenames

def parse_record(record):
    """
    input: a tfrecord
    output: parsed tfrecord based on features
    """
    name_to_features = {
        'image/class/label': tf.io.FixedLenFeature([], tf.int64),
        'image/class/synset': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string)
    }
    return tf.io.parse_single_example(record, name_to_features)

def preprocess_data(is_training):
    """
    input: bool is_training
    output: training/val X,y tuple ready to feed into models
    """
    #init X and y list
    X = []
    y = []
    #get all filenames
    filenames = get_filenames(is_training)
    raw_dataset = tf.data.TFRecordDataset(filenames)
    parsed_dataset = raw_dataset.map(parse_record)
    num_samples = None
    if is_training:
        num_samples = num_train_files*buffer_size
    else:
        num_samples = num_val_files*buffer_size
        
    print(num_samples)
    #get image and label from each parsed sample
    for parsed in parsed_dataset.take(num_samples):
        image = tf.io.decode_jpeg(parsed['image/encoded'], channels=3)
        label = parsed["image/class/label"]
        X.append(image)
        y.append(label)
    #reshape X to num_samples * num_channel * height * width 
    #cast to float 32
    #X = tf.image.convert_image_dtype(X, dtype=tf.float32, saturate=False)
    X = tf.reshape(tf.stack(X), (num_samples, num_channels, img_size, img_size))
    X = X/255
    #X = tf.cast(tf.reshape(tf.stack(X), 
                   #(num_samples, num_channels, img_size, img_size)),
                #tf.float32)
    y = tf.stack(y)
    #y = tf.cast(tf.stack(y), tf.float32)
    return X, y


In [29]:
def format_image(image):
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.transpose(tf.reshape(image,(img_size, img_size, num_channels)), [2,0,1])
    image /= 255.
    return image

def read_parsed(parsed):
    image = format_image(parsed['image/encoded'])
    label = parsed['image/class/label'] - 1
    
    return image, label
    
def get_dataset(filenames, batch_size=32):
    
    raw_dataset = tf.data.TFRecordDataset(filenames)
    parsed_dataset = raw_dataset.map(parse_record)
    dataset = parsed_dataset.map(read_parsed)
    
    dataset = dataset.batch(batch_size)
    
    return dataset

In [9]:
def train():
    is_training = True
    X_train, y_train = preprocess_data(is_training)
    model = netf.fishnet99()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x=X_train, y=y_train, batch_size=128, epochs=1, verbose=1, validation_split=0.2, shuffle=True)
    return "Done!"

In [10]:
# tf.config.run_functions_eagerly(True)
# train()

12800


ValueError: in user code:

    /mnt/c/StudyResources/Columbia/ECBME4040/envTF24/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /mnt/c/StudyResources/Columbia/ECBME4040/e4040-2021fall-project-fnet-cc4766-jw4167-jw4168/model_tf2/fishnet.py:226 call  *
        score = self.fish(x)
    /mnt/c/StudyResources/Columbia/ECBME4040/e4040-2021fall-project-fnet-cc4766-jw4167-jw4168/model_tf2/fishnet.py:188 call  *
        return self._fish_forward(all_feat)
    /mnt/c/StudyResources/Columbia/ECBME4040/e4040-2021fall-project-fnet-cc4766-jw4167-jw4168/model_tf2/fishnet.py:154 stage_forward  *
        tail_blk = tf.keras.Sequential(blks[:2])
    /mnt/c/StudyResources/Columbia/ECBME4040/envTF24/lib/python3.9/site-packages/tensorflow/python/training/tracking/base.py:530 _method_wrapper  **
        result = method(self, *args, **kwargs)
    /mnt/c/StudyResources/Columbia/ECBME4040/envTF24/lib/python3.9/site-packages/keras/engin

In [30]:
is_training = True
filenames = get_filenames(is_training)
train_ds = get_dataset(filenames)

val_filenames = get_filenames(False)
val_ds = get_dataset(val_filenames)

i = 0
for x in train_ds:
    i += 1
print(x)
print(i)

i = 0
for x in val_ds:
    i += 1
print(x)
print(i)

(<tf.Tensor: shape=(32, 3, 64, 64), dtype=float32, numpy=
array([[[[0.        , 0.02352941, 0.        , ..., 0.01176471,
          0.01176471, 0.02352941],
         [0.        , 0.        , 0.00784314, ..., 0.00392157,
          0.01176471, 0.00392157],
         [0.04313726, 0.        , 0.03921569, ..., 0.02352941,
          0.01960784, 0.        ],
         ...,
         [0.93333334, 0.9372549 , 1.        , ..., 0.92941177,
          0.92941177, 0.9372549 ],
         [0.        , 0.        , 0.04705882, ..., 0.        ,
          0.00784314, 0.02352941],
         [0.        , 0.        , 0.01176471, ..., 0.00392157,
          0.00392157, 0.00392157]],

        [[0.01960784, 0.07058824, 0.00392157, ..., 0.        ,
          0.        , 0.00392157],
         [0.01568628, 0.01176471, 0.02352941, ..., 0.        ,
          0.        , 0.        ],
         [0.07058824, 0.01176471, 0.05098039, ..., 0.02352941,
          0.01176471, 0.        ],
         ...,
         [0.96862745, 0.945098

(<tf.Tensor: shape=(16, 3, 64, 64), dtype=float32, numpy=
array([[[[0.31764707, 0.34117648, 0.34901962, ..., 0.27058825,
          0.44313726, 0.4392157 ],
         [0.29803923, 0.33333334, 0.3529412 , ..., 0.34509805,
          0.4862745 , 0.45490196],
         [0.31764707, 0.34509805, 0.3529412 , ..., 0.40784314,
          0.22745098, 0.32941177],
         ...,
         [0.32156864, 0.32156864, 0.3137255 , ..., 0.3882353 ,
          0.39607844, 0.28235295],
         [0.35686275, 0.33333334, 0.30980393, ..., 0.38431373,
          0.3372549 , 0.2627451 ],
         [0.35686275, 0.32941177, 0.3137255 , ..., 0.41568628,
          0.34901962, 0.3647059 ]],

        [[0.3882353 , 0.4117647 , 0.41960785, ..., 0.25490198,
          0.4117647 , 0.4117647 ],
         [0.36862746, 0.40392157, 0.42352942, ..., 0.32156864,
          0.45490196, 0.42745098],
         [0.3882353 , 0.41568628, 0.42352942, ..., 0.38431373,
          0.19607843, 0.29411766],
         ...,
         [0.3764706 , 0.384313

In [31]:
input_shape = (3, 64, 64)
output_size = 200

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=input_shape, 
                                data_format="channels_first"))
model.add(tf.keras.layers.MaxPooling2D((2,2), data_format="channels_first"))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation="relu",
                                data_format="channels_first"))
model.add(tf.keras.layers.MaxPooling2D((2,2), data_format="channels_first"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation="relu"))
model.add(tf.keras.layers.Dense(output_size, activation="softmax"))

model.summary()



Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_102 (Conv2D)          (None, 32, 62, 62)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 31, 31)        0         
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 64, 29, 29)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 14, 14)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              12846080  
_________________________________________________________________
dense_3 (Dense)              (None, 200)             

In [32]:
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, epochs=2)

Epoch 1/2


InvalidArgumentError:  Default MaxPoolingOp only supports NHWC on device type CPU
	 [[node sequential_29/max_pooling2d_4/MaxPool (defined at <ipython-input-32-08171e16969b>:5) ]] [Op:__inference_train_function_531014]

Function call stack:
train_function


# Loading by ImageDataGenerator

In [7]:
!ls 

FishNet_Pytorch     README.md	    data		main.py
FishNet_Tensorflow  Untitled.ipynb  data_preprocessing	model_tf2


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1/255., dtype=tf.float32, data_format="channels_first")

train_generator = datagen.flow_from_directory(
    'data_preprocessing/data/tiny-imagenet-200/train',
    target_size=(64,64),
    batch_size=32)


Found 100000 images belonging to 200 classes.


In [9]:
model = netf.myfishnet()
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_generator, batch_size=32, epochs=2)


UnimplementedError: The Conv2D op currently only supports the NHWC tensor format on the CPU. The op was given the format: NCHW [Op:Conv2D]